In [1]:
import csv
from datetime import datetime

In [2]:
class CSVWriter():
    def __init__(self, source, destination):
        self.i = 0
        self.source = source
        self.destination = destination
        with open(self.source, mode='r', newline='') as src, open(self.destination, mode='w', newline='') as dest:
            reader = csv.DictReader(src)
            csv.DictWriter(dest, fieldnames=reader.fieldnames).writeheader()

    def writeLine(self):
        lineWritten = 0
        with open(self.source, mode='r', newline='') as src, open(self.destination, mode='a', newline='') as dest:
            reader = csv.DictReader(src)
            writer = csv.DictWriter(dest, fieldnames=reader.fieldnames)
            for index, row in enumerate(reader):
                if index==self.i:
                    writer.writerow(row)
                    lineWritten = 1
        self.i = self.i+1
        if lineWritten:
            return {"message": "Data updated successfully"}
        else:
            return {"message": "Reached end of source file"}
        


In [101]:
class Stock():
    def __init__(self, name, latest):
        self.name = name
        self.closing_value = 1E6 # Initialize a large number, ensures initial percent is more or less 0 and avoids division by zero
        self.latest = latest
        #self.updated_today = 0
        self.last_updated = datetime(year=2016, month=12, day=31).date()
        self.percent = self.latest/self.closing_value*100

    def __str__(self): # Simple 'print' function for easy debugging
        return f'Stock: [name: {self.name}, closing_value: {self.closing_value}, latest: {self.latest},\n last_updated: {self.last_updated}, percent: {self.percent:.2f}]'

    def calc_percent(self):
        return self.latest/self.closing_value*100-100
    
    def to_dict(self, *args):
        return {key: getattr(self, key, None) for key in args}
    
def get_winners(stocks, number_of_winners=3):
    sorted_stocks = sorted(stocks, key=lambda x: x.percent, reverse=True)
    winners = []
    for ind, win in enumerate(sorted_stocks[:number_of_winners]):
        tmp_dict = {}
        tmp_dict['rank'] = ind+1
        tmp_dict.update(win.to_dict("name", "latest", "percent"))
        winners.append(tmp_dict)
        #print(win.__dict__)
    return winners

In [38]:
def get_filedata(source):
    data = []
    stocks = []
    with open(source, mode="r") as file:
        reader = csv.DictReader(file, delimiter=';')
        for row in reader:
            # Add check for the stocks here, if new kod add to 'stocks', otherwise update that stock
            datetime_obj = datetime.strptime(row['Date'], "%Y-%m-%d %H:%M:%S")
            row['twice'] = 2*int(row['Kurs'])
            row['Day'] = datetime_obj.day
            data.append(row)
        #headers = next(reader)  # Skip headers if needed
    return data

In [103]:
data = []
stocks = []
with open("Data.csv", mode="r") as file:
    reader = csv.DictReader(file, delimiter=';')
    for row in reader:
        datetime_obj = datetime.strptime(row['Date'], "%Y-%m-%d %H:%M:%S")
        #row['twice'] = 2*int(row['Kurs'])
        #row['Day'] = datetime_obj.day
        #unique_stocks = {stock.name for stock in stocks}
        new_stock = 1 # Track if new stock, if yes add it to stocks
        for stock in stocks:
            if row['Kod'] == stock.name:
                if datetime_obj.date()>stock.last_updated:
                    stock.closing_value = stock.latest
                    stock.last_updated = datetime_obj.date()
                stock.latest = int(row['Kurs'])
                stock.percent = stock.calc_percent()
                new_stock = 0
                print(f"Updated {stock.name} value to {stock.latest}")
        if new_stock:
            stocks.append(Stock(name=row['Kod'], latest=int(row['Kurs'])))
        #if row['Kod'] in unique_stocks:
        #    a = 0
        #else:
        #    stocks.append(Stock(name=row['Kod'], latest=int(row['Kurs'])))
        data.append(row)
        #stocks.append(Stock(row['Kod']))
winners = get_winners(stocks)

Updated ABB value to 218
Updated NCC value to 123
Updated NCC value to 121
Updated NCC value to 121
Updated AddLife B value to 27
Updated NCC value to 119
Updated ABB value to 219
Updated ABB value to 222
Updated NCC value to 117
Updated NCC value to 116
Updated 8TRA value to 225
Updated SSAB B value to 209
Updated AddLife B value to 38
Updated NCC value to 116
Updated NCC value to 118
Updated NCC value to 121


In [96]:
print(len(stocks))
for stock in stocks:
    print(stock)
#print(stocks[0])
unique_stocks = {stock.name for stock in stocks}
list_ = []
uni_list = {stock.name for stock in list_}
print(row)
print(unique_stocks)
print(uni_list)
print(data)

if 'ABB' in unique_stocks:
    print("A")
if 'ABB' in uni_list:
    print("B")

5
Stock: [name: ABB, closing_value: 219, latest: 222,
 last_updated: 2017-01-02, percent: 1.37]
Stock: [name: NCC, closing_value: 119, latest: 121,
 last_updated: 2017-01-02, percent: 1.68]
Stock: [name: AddLife B, closing_value: 27, latest: 38,
 last_updated: 2017-01-02, percent: 40.74]
Stock: [name: SSAB B, closing_value: 221, latest: 209,
 last_updated: 2017-01-02, percent: -5.43]
Stock: [name: 8TRA, closing_value: 226, latest: 225,
 last_updated: 2017-01-02, percent: -0.44]
{'Date': '2017-01-02 12:03:09', 'Kod': 'NCC', 'Kurs': '121'}
{'AddLife B', '8TRA', 'ABB', 'SSAB B', 'NCC'}
set()
[{'Date': '2017-01-01 12:00:00', 'Kod': 'ABB', 'Kurs': '217'}, {'Date': '2017-01-01 12:00:01', 'Kod': 'NCC', 'Kurs': '122'}, {'Date': '2017-01-01 12:00:02', 'Kod': 'ABB', 'Kurs': '218'}, {'Date': '2017-01-01 12:00:03', 'Kod': 'NCC', 'Kurs': '123'}, {'Date': '2017-01-01 12:00:04', 'Kod': 'NCC', 'Kurs': '121'}, {'Date': '2017-01-01 12:00:05', 'Kod': 'AddLife B', 'Kurs': '21'}, {'Date': '2017-01-01 12:00

In [100]:
def get_winners(stocks, number_of_winners=3):
    sorted_stocks = sorted(stocks, key=lambda x: x.percent, reverse=True)
    winners = []
    for ind, win in enumerate(sorted_stocks[:number_of_winners]):
        tmp_dict = {}
        tmp_dict['rank'] = ind+1
        tmp_dict.update(win.to_dict("name", "latest", "percent"))
        winners.append(tmp_dict)
        #print(win.__dict__)
    return winners

winners = get_winners(stocks)
for w in winners:
    print(w)
#print(winners)


{'rank': 1, 'name': 'AddLife B', 'latest': 38, 'percent': 40.74074074074073}
{'rank': 2, 'name': 'NCC', 'latest': 121, 'percent': 1.680672268907557}
{'rank': 3, 'name': 'ABB', 'latest': 222, 'percent': 1.3698630136986338}


In [57]:
a = datetime(year=2017, month=1, day=2)
b = datetime(year=2016, month=12, day=31)
print(a.date())
print(datetime(year=2016, month=12, day=31).date())
print(datetime_obj.date())
print(datetime_obj.date()>b.date())
print(datetime_obj.date()==a.date())

2017-01-02
2016-12-31
2017-01-02
True
True


In [ ]:

if row['Kod'] in unique_stocks:
    print("HEJ")
row.update(Kod='AAA')
if row['Kod'] in unique_stocks:
    print("HEJ")

In [19]:
print(isinstance(row, dict))

True
